# GOTM-ERSEM Plotting
**Author: Jun Sasaki  coded on 2024-09-16  Updated on 2024-09-16**<br>


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def update_datetime_in_file(input_file, start_time, end_time, freq, output_file):
    # 指定された期間で新しい時間範囲を生成
    time_range = pd.date_range(start=start_time, end=end_time, freq=freq)
    
    # 元のファイルを読み込む
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # 新しいファイルに書き込む
    with open(output_file, 'w') as f:
        for i, line in enumerate(lines):
            # 新しい日時を取得し、既存の行と組み合わせて書き込む
            value = line.split()[2]  # 元のデータの3列目（数値部分）を取得
            new_time = time_range[i]  # 新しい時間
            f.write(f"{new_time} {value}\n")

In [ ]:
'''
# File paths
input_file = 'light2_2016.dat'
output_file = 'updated_light2_2016.dat'

# Running the function
update_datetime_in_file(input_file, '2016-01-01 00:00:00', '2017-01-01 00:00:00', '1h', output_file)
'''

In [ ]:
def set_size(w, h, ax=None, colorbar=True):
    """ w, h: width, height in inches (size of plot area, excluding labels)
    """
    if ax is None:
        ax = plt.gca()

    # Get the position of the plot area excluding labels
    pos = ax.get_position()
    plot_width = pos.width
    plot_height = pos.height

    # Subplot parameters related to figure-wide margins (left, right, top, bottom)
    fig_left = ax.figure.subplotpars.left
    fig_right = ax.figure.subplotpars.right
    fig_top = ax.figure.subplotpars.top
    fig_bottom = ax.figure.subplotpars.bottom

    # Calculate the figure width and height based on the plot area
    fig_width = w / plot_width
    fig_height = h / plot_height
    ax.figure.set_size_inches(fig_width, fig_height)

In [ ]:
ncfile = 'Kawasaki_2016_hourly_mean_16.06.nc'
ds = xr.open_dataset(ncfile)
#ds = ds.sel(time=slice('2016-01-01', '2016-01-03'))

In [ ]:
ds

In [ ]:
var='rho' # 'light_EIR' 'O2_o'
vts = ds[var].squeeze(dim=['lat', 'lon'], drop=True)
depth = ds['z'].squeeze(dim=['lat', 'lon'], drop=True)
time = vts['time'].values
depth_values = depth.values
time_edges = np.concatenate([time, [time[-1] + (time[-1] - time[-2])]]) - (time[1] - time[0]) / 2
depth_edges = np.concatenate([depth_values[0, :], [depth_values[0, -1] + (depth_values[0, -1] - depth_values[0, -2])]]) - (depth_values[0, 1] - depth_values[0, 0]) / 2

In [ ]:
#vts[1, 0:99]

In [ ]:
vmin, vmax = (1010, 1026) # (12,30) (20,35) (0,600)
cbar_label='Density (kg/m$_3$)' # 'Radiation (W/m$^2$)' # 'Salt. (PSU)' 'O$_2$ (mmol/m$^3$)'
fig, ax = plt.subplots()
mesh=ax.pcolormesh(time_edges, depth_edges, vts.T, shading='flat', cmap='jet', vmin=vmin, vmax=vmax)
ax.set_xlabel('')
ax.set_ylabel('Depth (m)', fontsize=12)
#cax = fig.add_subplot(gs[0,1])
cbar = fig.colorbar(mesh, ax=ax, pad=0.01)
cbar.set_label(cbar_label)
ax.tick_params(axis='x', rotation=45)
set_size(8,1.6,ax=ax, colorbar=False)
plt.savefig('test1.png', dpi=600, bbox_inches='tight')
